In [1]:
import pandas as pd
import os
dir = 'data/features'
#dir = 'data/features_sample'
train = pd.read_pickle(os.path.join(dir, 'train.pkl'))
valid = pd.read_pickle(os.path.join(dir, 'valid.pkl'))
test = pd.read_pickle(os.path.join(dir, 'test.pkl'))

In [18]:
import numpy as np

col_in_use = [
    'num_morphemes',
    'negative',
    'self_ref',
    'cognitive',
    'hedges',
    'hesitations',
    'exlusion_words',
    'negations',
    'motion_words',
    'sense_words',
    'noun',
    'verb',
    'particle',
    'adjective',
    'conjunction',
    'connections',
]

def make_data(df, type='sum'):
    X, y = [], []
    for _, row in df.iterrows():
        _X, _y = _make_data(row, type=type)
        X.append(_X)
        y.append(_y)
    return X, y

def _make_data(row, type='sum'):
    if type=='sum':
        X = [row[f'{col}_sum'] for col in col_in_use]
        X.append(row['TTR'])
    elif type=='mean':
        X = [row[f'{col}_mean'] for col in col_in_use]
        X.append(row['TTR'])
    elif type=='std':
        X = [row[f'{col}_std'] for col in col_in_use]
        X.append(row['TTR'])
    elif type=='all':
        X = [row[f'{col}_sum'] for col in col_in_use]
        X.extend([row[f'{col}_mean'] for col in col_in_use])
        X.extend([row[f'{col}_std'] for col in col_in_use])
        X.append(row['TTR'])
    y = row.labels
    return X, y

In [19]:
type='sum'
train_X, train_y = make_data(train, type=type)
valid_X, valid_y = make_data(valid, type=type)
train_X.extend(valid_X), train_y.extend(valid_y)
test_X, test_y = make_data(test, type=type)

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

forest = RandomForestClassifier(random_state=1).fit(train_X, train_y)

In [24]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_recall_fscore_support
import pandas as pd
import os

save_dir = ('outputs/nested/randomforest')
os.makedirs(save_dir, exist_ok=True)

predict = forest.predict(test_X)
acc = accuracy_score(test_y, predict)
c_report = classification_report(test_y, predict)
cm = pd.DataFrame(confusion_matrix(test_y, predict))
scores_df = pd.DataFrame(np.array(precision_recall_fscore_support(test_y, predict)).T,
            columns=["precision", "recall", "f1", "support"],
        )

print(acc)
print(cm)
print(c_report)
print(scores_df)

with open(os.path.join(save_dir, 'classification_report.txt'), 'w') as f:
        f.write(str(acc)+'\n')
        f.write(c_report)
cm.to_csv(os.path.join('cm.csv'))
scores_df.to_csv(os.path.join(save_dir, 'scores.csv'))

0.5237704918032787
      0    1
0  5566  534
1  5276  824

   precision    recall        f1  support
0   0.513374  0.912459  0.657065   6100.0
1   0.606775  0.135082  0.220971   6100.0
